# Creating a logistic regression to predict absenteeism

## Import the relevant libraries

In [156]:
# import the relevant libraries
import pandas as pd
import numpy as np

## Load the data

In [157]:
# load the preprocessed CSV data
data_preprocessed = pd.read_csv('Absenteeism_new_data.csv')

In [158]:
# eyeball the data
data_preprocessed.head()

,ID,Reason for Absence,Date,Transportation Expense,Distance to Work,Age,Daily Work Load Average,Body Mass Index,Education,Children,Pets
0,22,27,01/06/2018,179,26,30,237.656,19,3,0,0
1,10,7,04/06/2018,361,52,28,237.656,27,1,1,4
2,14,23,06/06/2018,155,12,34,237.656,25,1,2,0
3,17,25,08/06/2018,179,22,40,237.656,22,2,2,0
4,14,10,08/06/2018,155,12,34,237.656,25,1,2,0


In [159]:
# convert the 'Date' column into datetime
data_preprocessed['Date'] = pd.to_datetime(data_preprocessed['Date'], format='%d/%m/%Y')

# create a list with month values retrieved from the 'Date' column
list_months = []
for i in range(data_preprocessed.shape[0]):
    list_months.append(data_preprocessed['Date'][i].month)

# insert the values in a new column in df, called 'Month Value'
data_preprocessed['Month Value'] = list_months

# create a new feature called 'Day of the Week'
data_preprocessed['Day of the Week'] = data_preprocessed['Date'].apply(lambda x: x.weekday())


# drop the 'Date' column from df
data_preprocessed = data_preprocessed.drop(['Date'], axis = 1)

## A comment on the targets

In [144]:
# create a checkpoint by dropping the unnecessary variables
# also drop the variables we 'eliminated' after exploring the weights
data_preprocessed.drop(['Daily Work Load Average','Distance to Work'],axis=1)

,ID,Reason for Absence,Date,Transportation Expense,Age,Body Mass Index,Education,Children,Pets
0,22,27,01/06/2018,179,30,19,3,0,0
1,10,7,04/06/2018,361,28,27,1,1,4
2,14,23,06/06/2018,155,34,25,1,2,0
3,17,25,08/06/2018,179,40,22,2,2,0
4,14,10,08/06/2018,155,34,25,1,2,0
5,28,11,11/06/2018,225,28,24,1,1,2
6,16,7,13/06/2018,118,46,25,1,2,0
7,22,27,13/06/2018,179,30,19,3,0,0
8,34,26,15/06/2018,118,37,28,1,0,0
9,34,10,20/06/2018,118,37,28,1,0,0


## Select the inputs for the regression

In [145]:
# Create a variable that will contain the inputs (everything without the targets)
unscaled_inputs = data_preprocessed

## Standardize the data

In [146]:
# import the libraries needed to create the Custom Scaler
# note that all of them are a part of the sklearn package
# moreover, one of them is actually the StandardScaler module, 
# so you can imagine that the Custom Scaler is build on it

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler

# create the Custom Scaler class

class CustomScaler(BaseEstimator,TransformerMixin): 
    
    # init or what information we need to declare a CustomScaler object
    # and what is calculated/declared as we do
    
    def __init__(self,columns,copy=True,with_mean=True,with_std=True):
        
        # scaler is nothing but a Standard Scaler object
        self.scaler = StandardScaler(copy,with_mean,with_std)
        # with some columns 'twist'
        self.columns = columns
        self.mean_ = None
        self.var_ = None
        
    
    # the fit method, which, again based on StandardScale
    
    def fit(self, X, y=None):
        self.scaler.fit(X[self.columns], y)
        self.mean_ = np.mean(X[self.columns])
        self.var_ = np.var(X[self.columns])
        return self
    
    # the transform method which does the actual scaling

    def transform(self, X, y=None, copy=None):
        
        # record the initial order of the columns
        init_col_order = X.columns
        
        # scale all features that you chose when creating the instance of the class
        X_scaled = pd.DataFrame(self.scaler.transform(X[self.columns]), columns=self.columns)
        
        # declare a variable containing all information that was not scaled
        X_not_scaled = X.loc[:,~X.columns.isin(self.columns)]
        
        # return a data frame which contains all scaled features and all 'not scaled' features
        # use the original order (that you recorded in the beginning)
        return pd.concat([X_not_scaled, X_scaled], axis=1)[init_col_order]

In [147]:
# check what are all columns that we've got
unscaled_inputs.columns.values

array(['ID', 'Reason for Absence', 'Date', 'Transportation Expense',
       'Distance to Work', 'Age', 'Daily Work Load Average',
       'Body Mass Index', 'Education', 'Children', 'Pets'], dtype=object)

In [148]:
# choose the columns to scale
# we later augmented this code and put it in comments
# columns_to_scale = ['Month Value','Day of the Week', 'Transportation Expense', 'Distance to Work',
       #'Age', 'Daily Work Load Average', 'Body Mass Index', 'Children', 'Pet']
    
# select the columns to omit
columns_to_omit = ['Reason_1', 'Reason_2', 'Reason_3', 'Reason_4','Education']

In [149]:
# create the columns to scale, based on the columns to omit
# use list comprehension to iterate over the list
columns_to_scale = [x for x in unscaled_inputs.columns.values if x not in columns_to_omit]

In [150]:
# declare a scaler object, specifying the columns you want to scale
absenteeism_scaler = CustomScaler(columns_to_scale)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:71: FutureWarning: Pass copy=True, with_mean=True, with_std=True as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


In [151]:
# fit the data (calculate mean and standard deviation); they are automatically stored inside the object 
absenteeism_scaler.fit(unscaled_inputs)

ValueError: could not convert string to float: '01/06/2018'

In [ ]:
# standardizes the data, using the transform method 
# in the last line, we fitted the data - in other words
# we found the internal parameters of a model that will be used to transform data. 
# transforming applies these parameters to our data
# note that when you get new data, you can just call 'scaler' again and transform it in the same way as now
scaled_inputs = absenteeism_scaler.transform(unscaled_inputs)

In [ ]:
# the scaled_inputs are now an ndarray, because sklearn works with ndarrays
scaled_inputs

In [ ]:
# check the shape of the inputs
scaled_inputs.shape

## Split the data into train & test and shuffle

### Import the relevant module

In [ ]:
# import train_test_split so we can split our data into train and test
from sklearn.model_selection import train_test_split

### Split

In [ ]:
# check how this method works
train_test_split(scaled_inputs, targets)

In [ ]:
# declare 4 variables for the split
x_train, x_test, y_train, y_test = train_test_split(scaled_inputs, targets, #train_size = 0.8, 
                                                                            test_size = 0.2, random_state = 20)

In [ ]:
# check the shape of the train inputs and targets
print (x_train.shape, y_train.shape)

In [ ]:
# check the shape of the test inputs and targets
print (x_test.shape, y_test.shape)

## Logistic regression with sklearn

In [ ]:
# import the LogReg model from sklearn
from sklearn.linear_model import LogisticRegression

# import the 'metrics' module, which includes important metrics we may want to use
from sklearn import metrics

### Training the model

In [ ]:
# create a logistic regression object
reg = LogisticRegression()

In [ ]:
# fit our train inputs
# that is basically the whole training part of the machine learning
reg.fit(x_train,y_train)

In [ ]:
# assess the train accuracy of the model
reg.score(x_train,y_train)

### Manually check the accuracy

In [ ]:
# find the model outputs according to our model
model_outputs = reg.predict(x_train)
model_outputs

In [ ]:
# compare them with the targets
y_train

In [ ]:
# ACTUALLY compare the two variables
model_outputs == y_train

In [ ]:
# find out in how many instances we predicted correctly
np.sum((model_outputs==y_train))

In [ ]:
# get the total number of instances
model_outputs.shape[0]

In [ ]:
# calculate the accuracy of the model
np.sum((model_outputs==y_train)) / model_outputs.shape[0]

### Finding the intercept and coefficients

In [ ]:
# get the intercept (bias) of our model
reg.intercept_

In [ ]:
# get the coefficients (weights) of our model
reg.coef_

In [ ]:
# check what were the names of our columns
unscaled_inputs.columns.values

In [ ]:
# save the names of the columns in an ad-hoc variable
feature_name = unscaled_inputs.columns.values

In [ ]:
# use the coefficients from this table (they will be exported later and will be used in Tableau)
# transpose the model coefficients (model.coef_) and throws them into a df (a vertical organization, so that they can be
# multiplied by certain matrices later) 
summary_table = pd.DataFrame (columns=['Feature name'], data = feature_name)

# add the coefficient values to the summary table
summary_table['Coefficient'] = np.transpose(reg.coef_)

# display the summary table
summary_table

In [ ]:
# do a little Python trick to move the intercept to the top of the summary table
# move all indices by 1
summary_table.index = summary_table.index + 1

# add the intercept at index 0
summary_table.loc[0] = ['Intercept', reg.intercept_[0]]

# sort the df by index
summary_table = summary_table.sort_index()
summary_table

## Interpreting the coefficients

In [ ]:
# create a new Series called: 'Odds ratio' which will show the.. odds ratio of each feature
summary_table['Odds_ratio'] = np.exp(summary_table.Coefficient)

In [ ]:
# display the df
summary_table

In [ ]:
# sort the table according to odds ratio
# note that by default, the sort_values method sorts values by 'ascending'
summary_table.sort_values('Odds_ratio', ascending=False)

## Testing the model

In [ ]:
# assess the test accuracy of the model
reg.score(x_test,y_test)

In [ ]:
# find the predicted probabilities of each class
# the first column shows the probability of a particular observation to be 0, while the second one - to be 1
predicted_proba = reg.predict_proba(x_test)

# let's check that out
predicted_proba

In [ ]:
predicted_proba.shape

In [ ]:
# select ONLY the probabilities referring to 1s
predicted_proba[:,1]

## Save the model

In [ ]:
# import the relevant module
import pickle

In [ ]:
# pickle the model file
with open('model', 'wb') as file:
    pickle.dump(reg, file)

In [ ]:
# pickle the scaler file
with open('scaler','wb') as file:
    pickle.dump(absenteeism_scaler, file)